In [2]:
# Initialize Tesseract OCR (this should be the legacy one with original LSTM)
import cv2
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [ ]:
# Setup stream for MJSynth datraset 
from datasets import load_dataset

ds = load_dataset("priyank-m/MJSynth_text_recognition", split='train', streaming=True)

c:\Users\alexn\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Test on first sample of MJSynth dataset
from PIL import Image
test = next(iter(ds))
display(test['image'])

iterable_dataset = ds.shuffle(seed=42, buffer_size=10)

In [6]:
# Set up EasyOCR engine
import easyocr

easyocr_reader = easyocr.Reader(['en'])

In [ ]:
# Set up Keras OCR engine
import keras_ocr

keras_pipeline = keras_ocr.pipeline.Pipeline()



Looking for C:\Users\alexn\.keras-ocr\craft_mlt_25k.h5

Instructions for updating:
Use `tf.image.resize(...method=ResizeMethod.BILINEAR...)` instead.

Looking for C:\Users\alexn\.keras-ocr\crnn_kurapan.h5


In [8]:
# Save 10000 samples of MJSynth data
# ONLY EXECUTE THE DATA EXTRACTION ONCE
# from PIL import Image
# import numpy as np
# sample_size = 10000

# ds_iter = iter(iterable_dataset)

# counter = 1
# lines = []
# for sample in ds_iter:
#     image_data = sample['image']
#     label_data = sample['label']
    
#     image_data.save(f'./pictures/{counter}.png', 'PNG')
#     counter += 1
#     lines.append(label_data + "\n")
    
#     if counter == sample_size + 1:
#         break
    
# with open("label.txt", 'w') as file:
#     file.writelines(lines)
    
# Read labels
labels = None
with open('label.txt', 'r') as file:
    labels = file.readlines()
    

In [9]:
# Function to compute true positives, false positives, and false negatives given prediction and ground truth
# Sklearn.metrics did not work
from collections import Counter

def compute_tp_fp_fn(predicted, ground_truth):
    # Count character frequencies in both strings
    pred_count = Counter(predicted)
    gt_count = Counter(ground_truth)
    
    # Compute True Positives (correctly matched characters)
    true_positives = sum((pred_count & gt_count).values())
    
    # Compute False Positives (extra characters in predicted text)
    false_positives = sum((pred_count - gt_count).values())
    
    # Compute False Negatives (missing characters in predicted text)
    false_negatives = sum((gt_count - pred_count).values())
    
    return true_positives, false_positives, false_negatives

In [9]:
# Test performance on all models (minus best Tesseract LSTM model) on 2000 samples of MJSynth dataset
import numpy as np

# Number of models to test = 6
amt_models = 6
amt_samples = 2000
correct_pred = np.zeros(amt_models)
incorrect_pred = np.zeros(amt_models)
total_precision_tess_legacy = 0
total_recall_tess_legacy = 0
total_f1_tess_legacy = 0
total_precision_tess_lstm = 0
total_recall_tess_lstm = 0
total_f1_tess_lstm = 0
total_precision_easyocr = 0
total_recall_easyocr = 0
total_f1_easyocr = 0
total_precision_keras = 0
total_recall_keras = 0
total_f1_keras = 0

for counter in range(1, amt_samples + 1):
    # Print which sample we are on for every 100 samples
    if (counter - 1) % 100 == 0:
        print(f"Working on Sample {counter}")
    
    # Read image + obtain text
    image_path = f'./pictures/{counter}.png'
    image = Image.open(image_path)
    label = labels[counter-1].strip().lower()
    
    # Prediction from Pytesseract Legacy Engine
    legacy_tesseract_result = pytesseract.image_to_string(image, lang='eng', config='--oem 0 --psm 6').strip().lower()
    
    # Prediction from Old Pytesseract lstm Model
    old_tesseract_lstm_result = pytesseract.image_to_string(image, lang='eng', config='--oem 1 --psm 6').strip().lower()

    # Prediction from EasyOCR
    easyocr_result = easyocr_reader.readtext(image_path)
    if easyocr_result:
        easyocr_result = easyocr_result[0][1].lower()
    else:
        easyocr_result = ""
    
    # Prediction from Keras OCR
    keras_image = keras_ocr.tools.read(image_path)
    keras_result = keras_pipeline.recognize([keras_image])
    if keras_result[0]:
        keras_result = keras_result[0][0][0].lower()
    else:
        keras_result = ""
    
    # Print results
    # print(f"Actual: {label}")
    # print(f"Legacy Tesseract Prediction: {legacy_tesseract_result}")
    # print(f"Old lstm Tesseract Prediction: {old_tesseract_lstm_result}")
    # print(f"EasyOCR Prediction: {easyocr_result}")
    # print(f"Keras OCR Prediction: {keras_result}")
    # print()
    
    # Record results (no case sensitivity)
    correct_pred[0] += (legacy_tesseract_result == label)
    incorrect_pred[0] += (legacy_tesseract_result != label)
    correct_pred[1] += (old_tesseract_lstm_result == label)
    incorrect_pred[1] += (old_tesseract_lstm_result != label)
    correct_pred[3] += (easyocr_result == label)
    incorrect_pred[3] += (easyocr_result != label)
    correct_pred[4] += (keras_result == label)
    incorrect_pred[4] += (keras_result != label)
    
    # Calculate characeter wise confusion matrix for Legacy Tesseract model
    tps_tess_legacy, fps_tess_legacy, fns_tess_legacy = compute_tp_fp_fn(legacy_tesseract_result, label)
    
    # Compute precision, recall, and f1 for Tesseract model
    precision_tess_legacy = recall_tess_legacy = f1_score_tess_legacy = 0
    
    if (tps_tess_legacy + fps_tess_legacy > 0):
        precision_tess_legacy = tps_tess_legacy / (tps_tess_legacy + fps_tess_legacy)
        
    if (tps_tess_legacy + fns_tess_legacy > 0):
        recall_tess_legacy = tps_tess_legacy / (tps_tess_legacy + fns_tess_legacy)
        
    if (precision_tess_legacy + recall_tess_legacy > 0):
        f1_score_tess_legacy = 2 * (precision_tess_legacy * recall_tess_legacy) / (precision_tess_legacy + recall_tess_legacy)
        
    # Aggregate results for Tesseract model
    total_precision_tess_legacy += precision_tess_legacy
    total_recall_tess_legacy += recall_tess_legacy
    total_f1_tess_legacy += f1_score_tess_legacy
    
    # Calculate characeter wise confusion matrix for Old lstm Tesseract model
    tps_tess_lstm, fps_tess_lstm, fns_tess_lstm = compute_tp_fp_fn(old_tesseract_lstm_result, label)
    
    # Compute precision, recall, and f1 for Tesseract model
    precision_tess_lstm = recall_tess_lstm = f1_score_tess_lstm = 0
    
    if (tps_tess_lstm + fps_tess_lstm> 0):
        precision_tess_lstm = tps_tess_lstm/ (tps_tess_lstm + fps_tess_lstm)
        
    if (tps_tess_lstm + fns_tess_lstm > 0):
        recall_tess_lstm = tps_tess_lstm/ (tps_tess_lstm + fns_tess_lstm)
        
    if (precision_tess_lstm + recall_tess_lstm > 0):
        f1_score_tess_lstm = 2 * (precision_tess_lstm * recall_tess_lstm) / (precision_tess_lstm+ recall_tess_lstm)
            
    # Aggregate results for Old lstmTesseract model
    total_precision_tess_lstm += precision_tess_lstm
    total_recall_tess_lstm += recall_tess_lstm
    total_f1_tess_lstm += f1_score_tess_lstm
    
    # Calculate characeter wise confusion matrix for EasyOCR model
    tps_easyocr, fps_easyocr, fns_easyocr = compute_tp_fp_fn(easyocr_result, label)
    
    # Compute precision, recall, and f1 for EasyOCR model
    precision_easyocr = recall_easyocr = f1_score_easyocr = 0
    
    if (tps_easyocr + fps_easyocr > 0):
        precision_easyocr = tps_easyocr / (tps_easyocr + fps_easyocr)
        
    if (tps_easyocr + fns_easyocr > 0):
        recall_easyocr = tps_easyocr / (tps_easyocr + fns_easyocr)
        
    if (precision_easyocr + recall_easyocr > 0):
        f1_score_easyocr = 2 * (precision_easyocr * recall_easyocr) / (precision_easyocr + recall_easyocr)
            
    # Aggregate results for EasyOCR model
    total_precision_easyocr += precision_easyocr
    total_recall_easyocr += recall_easyocr
    total_f1_easyocr += f1_score_easyocr
    
    # Calculate characeter wise confusion matrix for Keras OCR model
    tps_keras, fps_keras, fns_keras = compute_tp_fp_fn(keras_result, label)
    
    # Compute precision, recall, and f1 for Tesseract model
    precision_keras = recall_keras = f1_score_keras = 0
    
    if (tps_keras + fps_keras > 0):
        precision_keras = tps_keras / (tps_keras + fps_keras)
        
    if (tps_keras + fns_keras > 0):
        recall_keras = tps_keras / (tps_keras + fns_keras)
        
    if (precision_keras + recall_keras > 0):
        f1_score_keras = 2 * (precision_keras * recall_keras) / (precision_keras + recall_keras)
            
    # Aggregate results for Keras OCR model
    total_precision_keras += precision_keras
    total_recall_keras += recall_keras
    total_f1_keras += f1_score_keras
    
# print(correct_pred, incorrect_pred)
# print(total_precision_tess_legacy / amt_samples, total_recall_tess_legacy / amt_samples, total_f1_tess_legacy / amt_samples)
# print(total_precision_tess_lstm / amt_samples, total_recall_tess_lstm/ amt_samples, total_f1_tess_lstm/ amt_samples)
# print(total_precision_easyocr / amt_samples, total_recall_easyocr / amt_samples, total_f1_easyocr / amt_samples)
# print(total_precision_keras / amt_samples, total_recall_keras / amt_samples, total_f1_keras / amt_samples)


Working on Sample 1
1/1 [==============================] - 0s 115ms/step
Working on Sample 101
1/1 [==============================] - 0s 120ms/step
Working on Sample 201
1/1 [==============================] - 0s 144ms/step
Working on Sample 301
1/1 [==============================] - 0s 105ms/step
Working on Sample 401
1/1 [==============================] - 0s 113ms/step
Working on Sample 501
1/1 [==============================] - 0s 109ms/step
Working on Sample 601
1/1 [==============================] - 0s 104ms/step
Working on Sample 701
1/1 [==============================] - 0s 98ms/step
Working on Sample 801
1/1 [==============================] - 0s 102ms/step
Working on Sample 901
1/1 [==============================] - 0s 151ms/step
Working on Sample 1001
1/1 [==============================] - 0s 183ms/step
Working on Sample 1101
1/1 [==============================] - 0s 126ms/step
Working on Sample 1201
1/1 [==============================] - 0s 171ms/step
Working on Sample 1301
1/

In [10]:
# Print predictions result
print(f"Old Legacy Tesseract Model")
print(f"Number of Correct Predictions: {correct_pred[0]}")
print(f"Number of Incorrect Predictions: {incorrect_pred[0]}")
print(f"Accuracy: {(correct_pred[0] / amt_samples):.5f}")
print(f"Character-wise Precision: {(total_precision_tess_legacy / amt_samples):.5f}")
print(f"Character-wise Recall: {(total_recall_tess_legacy / amt_samples):.5f}")
print(f"Character-wise F1-score: {(total_f1_tess_legacy / amt_samples):.5f}\n")

print(f"Old lstm Tesseract Model")
print(f"Number of Correct Predictions: {correct_pred[1]}")
print(f"Number of Incorrect Predictions: {incorrect_pred[1]}")
print(f"Accuracy: {(correct_pred[1] / amt_samples):.5f}")
print(f"Character-wise Precision: {(total_precision_tess_lstm/ amt_samples):.5f}")
print(f"Character-wise Recall: {(total_recall_tess_lstm/ amt_samples):.5f}")
print(f"Character-wise F1-score: {(total_f1_tess_lstm/ amt_samples):.5f}\n")

print(f"EasyOCR Model")
print(f"Number of Correct Predictions: {correct_pred[3]}")
print(f"Number of Incorrect Predictions: {incorrect_pred[3]}")
print(f"Accuracy: {(correct_pred[3] / amt_samples):.5f}")
print(f"Character-wise Precision: {(total_precision_easyocr / amt_samples):.5f}")
print(f"Character-wise Recall: {(total_recall_easyocr / amt_samples):.5f}")
print(f"Character-wise F1-score: {(total_f1_easyocr / amt_samples):.5f}\n")

print(f"Keras OCR Model")
print(f"Number of Correct Predictions: {correct_pred[4]}")
print(f"Number of Incorrect Predictions: {incorrect_pred[4]}")
print(f"Accuracy: {(correct_pred[4] / amt_samples):.5f}")
print(f"Character-wise Precision: {(total_precision_keras / amt_samples):.5f}")
print(f"Character-wise Recall: {(total_recall_keras / amt_samples):.5f}")
print(f"Character-wise F1-score: {(total_f1_keras / amt_samples):.5f}\n")

Old Legacy Tesseract Model
Number of Correct Predictions: 596.0
Number of Incorrect Predictions: 1404.0
Accuracy: 0.29800
Character-wise Precision: 0.63963
Character-wise Recall: 0.62470
Character-wise F1-score: 0.62692

Old lstm Tesseract Model
Number of Correct Predictions: 954.0
Number of Incorrect Predictions: 1046.0
Accuracy: 0.47700
Character-wise Precision: 0.73800
Character-wise Recall: 0.70921
Character-wise F1-score: 0.71434

EasyOCR Model
Number of Correct Predictions: 1094.0
Number of Incorrect Predictions: 906.0
Accuracy: 0.54700
Character-wise Precision: 0.85282
Character-wise Recall: 0.82210
Character-wise F1-score: 0.82976

Keras OCR Model
Number of Correct Predictions: 1593.0
Number of Incorrect Predictions: 407.0
Accuracy: 0.79650
Character-wise Precision: 0.96113
Character-wise Recall: 0.91186
Character-wise F1-score: 0.92692



In [12]:
# Execute same test on best Tesseract LSTM model
#   Required installating new Tessseract package to support best LSTM model
import pytesseract
import numpy as np

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Evalulate best lstmmodel
#   Note: Need to change tesseract files between best lstmmodel vs. legacy + worse lstmmodel
total_precision_best_lstm = 0
total_recall_best_lstm = 0
total_f1_best_lstm = 0

for counter in range(1, amt_samples + 1):
    # Print which sample we are on for every 100 samples
    if (counter - 1) % 100 == 0:
        print(f"Working on Sample {counter}")
    
    # Read image + obtain text
    image_path = f'./pictures/{counter}.png'
    image = Image.open(image_path)
    label = labels[counter-1].strip().lower()
        
    # Prediction from Best Pytesseract lstmModel
    best_lstm_tesseract_result = pytesseract.image_to_string(image, lang='eng', config='--oem 1 --psm 6').strip().lower()
    
    # Record results (no case sensitivity)
    correct_pred[2] += (best_lstm_tesseract_result == label)
    incorrect_pred[2] += (best_lstm_tesseract_result != label)
                
    # Calculate characeter wise confusion matrix for Best lstm Tesseract model
    tps_best_lstm, fps_best_lstm, fns_best_lstm = compute_tp_fp_fn(best_lstm_tesseract_result, label)
    
    # Compute precision, recall, and f1 for Best lstmTesseract model
    precision_best_lstm = recall_best_lstm = f1_score_best_lstm = 0
    
    if (tps_best_lstm + fps_best_lstm > 0):
        precision_best_lstm= tps_best_lstm/ (tps_best_lstm + fps_best_lstm)
        
    if (tps_best_lstm + fns_best_lstm> 0):
        recall_best_lstm = tps_best_lstm/ (tps_best_lstm + fns_best_lstm)
        
    if (precision_best_lstm + recall_best_lstm> 0):
        f1_score_best_lstm = 2 * (precision_best_lstm * recall_best_lstm) / (precision_best_lstm + recall_best_lstm)
            
    # Aggregate results for Best lstmTesseract model
    total_precision_best_lstm += precision_best_lstm
    total_recall_best_lstm += recall_best_lstm
    total_f1_best_lstm += f1_score_best_lstm


Working on Sample 1
Working on Sample 101
Working on Sample 201
Working on Sample 301
Working on Sample 401
Working on Sample 501
Working on Sample 601
Working on Sample 701
Working on Sample 801
Working on Sample 901
Working on Sample 1001
Working on Sample 1101
Working on Sample 1201
Working on Sample 1301
Working on Sample 1401
Working on Sample 1501
Working on Sample 1601
Working on Sample 1701
Working on Sample 1801
Working on Sample 1901


In [13]:
# Print predictions result for best LSTM Tesseract Model
print(f"Best lstm Tesseract Model")
print(f"Number of Correct Predictions: {correct_pred[2]}")
print(f"Number of Incorrect Predictions: {incorrect_pred[2]}")
print(f"Accuracy: {(correct_pred[2] / amt_samples):.5f}")
print(f"Character-wise Precision: {(total_precision_best_lstm/ amt_samples):.5f}")
print(f"Character-wise Recall: {(total_recall_best_lstm/ amt_samples):.5f}")
print(f"Character-wise F1-score: {(total_f1_best_lstm / amt_samples):.5f}\n")

Best lstm Tesseract Model
Number of Correct Predictions: 1188.0
Number of Incorrect Predictions: 812.0
Accuracy: 0.59400
Character-wise Precision: 0.85306
Character-wise Recall: 0.85821
Character-wise F1-score: 0.85299



In [14]:
# Engine for TTS 
import pyttsx3

def read_text_aloud(text):
    """
    Convert text to speech using pyttsx3.
    """
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

In [15]:
# Read sample text from Keras OCR engine predictions
for i in range(1, 6):
    image_path = f'./pictures/{i}.png'
    keras_image = keras_ocr.tools.read(image_path)
    keras_result = keras_pipeline.recognize([keras_image])
    if keras_result[0]:
        keras_result = keras_result[0][0][0].lower()
    else:
        keras_result = ""
        
    print(keras_result)
    read_text_aloud(keras_result)

1/1 [==============================] - 5s 5s/step
aseptic
1/1 [==============================] - 0s 113ms/step
set
1/1 [==============================] - 0s 91ms/step
yearbooks
1/1 [==============================] - 0s 82ms/step
teddies
1/1 [==============================] - 0s 113ms/step
chloral
